In [1]:
import numpy as np

In [2]:
def get_balanced_transportation_problem(supply, demand, costs):
    total_supply = sum(supply)
    total_demand = sum(demand)
    
    if total_supply < total_demand:
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs 
        return new_supply, demand, new_costs
    if total_supply > total_demand:
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        return supply, new_demand, new_costs
    return supply, demand, costs

In [3]:
supply = [2000,3000 ]
demand = [2200, 1700, 1400]
costs = [
    [12, 14,19],
    [7, 16, 12],
]
get_balanced_transportation_problem(supply, demand, costs)

([2000, 3000, 300], [2200, 1700, 1400], [[12, 14, 19], [7, 16, 12]])

In [4]:
supply = [2000,3000 ]
demand = [2200, 1700, 1400]
costs = [
    [12, 14,19],
    [7, 16, 12],
    [0,0,0]
]
get_balanced_transportation_problem(supply, demand, costs)

([2000, 3000, 300], [2200, 1700, 1400], [[12, 14, 19], [7, 16, 12], [0, 0, 0]])

## NorthWest Corner Method

In [5]:
def north_west_corner(supply, demand):
    """
    This method create copies of both the supply and demand
    """
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    i = 0
    j = 0
    bfs = []
    while len(bfs) < len(supply) + len(demand) - 1:
        """
        Verify if length of supply(m) + length of demand(n) - 1 is greater than the bfs
        """
        s = supply_copy[i]
        d = demand_copy[j]
        v = min(s, d)
        """
        Determines the minimum value between the supply and demand at a particular index
        """
        supply_copy[i] -= v
        demand_copy[j] -= v
        bfs.append(((i, j), v))
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
    return bfs

In [20]:
supply = [2000,3000 ]
demand = [2200, 1700, 1400]
bfs = north_west_corner(supply, demand)
print(bfs)

[((0, 0), 2000), ((1, 0), 200), ((1, 1), 1700), ((1, 2), 1100)]


## Stepping Stone

In [6]:
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: continue
                
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
            
    return us, vs 

In [7]:
def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

In [10]:
def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

In [12]:
def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

In [13]:
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row

In [14]:
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

In [15]:
def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

In [8]:
def transportation_northwest(supply, demand, costs):
    balanced_supply, balanced_demand, balanced_costs = get_balanced_transportation_problem(
        supply, demand, costs
    )
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, balanced_costs)
        ws = get_ws(bfs, balanced_costs, us, vs)
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        return bfs
    
    basic_variables = inner(north_west_corner(balanced_supply, balanced_demand))
    solution = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        solution[i][j] = v

    return solution

In [17]:
def get_total_cost(costs, solution):
    optimal_solution = 0
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            optimal_solution += cost * solution[i][j]
    return optimal_solution

In [18]:
supply = [700,900 ]
demand = [400, 650, 450,100]
costs = [
    [30, 25,29,0],
    [25, 33, 28,0],
    
]
solution = transportation_northwest(supply, demand, costs)
print(solution)
print('Optimal Solution: ', get_total_cost(costs, solution))

[[  0. 650.   0.  50.]
 [400.   0. 450.  50.]]
Optimal Solution:  38850.0
